# 필요 라이브러리 임포트

In [ ]:
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.prompts import PromptTemplate
import pandas as pd
import time
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import os
from langchain.llms import Ollama
from langchain.llms import OpenAI
from langchain_core.runnables import RunnablePassthrough

# 데이터프레임

In [ ]:
df = pd.read_csv('filtered_review.csv')
df.head()

,식당,리뷰
0,호반,Rating : 👍👍👍\nPlace : 낙원동 호반\n\n종로 3가 낙원상가 인근에...
1,호반,"여기 진짜 노포 찐맛집이네요. 병어조림을 먹었는데, 양념벨런스가 너무 좋고 입에서 ..."
2,호반,순대국밥에 들어있는 순대가 너무 특이하네요. 하얀색 대창??에 속에 두부가 들어있어...
3,호반,예전부터 여러 블러거들의 맛있다는 소문은 들었지만 우연찮게 지나다 들렀던 집.\n...
4,호반,"순대를 좋아하고, 노포를 좋아한다면 가볼만한 곳\n다만 가게가 크지 않아 1차보다는..."


# openai를 위한 키

In [ ]:
os.environ['OPENAI_API_KEY'] = ''

# RAG 사용

In [ ]:
# 저장해놓은 벡터 저장소 불러오기
food_openai_db = Chroma(
    persist_directory='food_openai_db',
    embedding_function=OpenAIEmbeddings()
)

## 정보 확인하기

In [ ]:
template = """
너는 서울뚝배기 식당의 리뷰들을 요약해주는 ai야.
리뷰들 요약해서 맛, 양, 가격, 서비스 부분에서 어떤지 줄글로 설명해줘.
"""

openai_retriever = food_openai_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'k':50, 'score_threshold': 0.7}
)

docs = openai_retriever.invoke(template)
docs

[Document(metadata={'row': 534, 'source': 'filtered_review.csv'}, page_content='식당: 서울맛집\n리뷰: 가격 서비스 맛  전부 환상입니다'),
 Document(metadata={'row': 4928, 'source': 'filtered_review.csv'}, page_content='식당: 서울맛집\n리뷰: 가격 서비스 맛  전부 환상입니다'),
 Document(metadata={'row': 483, 'source': 'filtered_review.csv'}, page_content='식당: 서울맛집\n리뷰: 가격에 비해 분위기도 좋고 맛도 좋고 사장님 서비스도 좋아요.'),
 Document(metadata={'row': 4877, 'source': 'filtered_review.csv'}, page_content='식당: 서울맛집\n리뷰: 가격에 비해 분위기도 좋고 맛도 좋고 사장님 서비스도 좋아요.'),
 Document(metadata={'row': 467, 'source': 'filtered_review.csv'}, page_content='식당: 서울맛집\n리뷰: 맛있어요! 가격 착해요! 더이상 설명 불가!'),
 Document(metadata={'row': 4861, 'source': 'filtered_review.csv'}, page_content='식당: 서울맛집\n리뷰: 맛있어요! 가격 착해요! 더이상 설명 불가!'),
 Document(metadata={'row': 4897, 'source': 'filtered_review.csv'}, page_content='식당: 서울맛집\n리뷰: 가격대비 맛있게 푸짐하게\n먹을수있는것 같아요.'),
 Document(metadata={'row': 503, 'source': 'filtered_review.csv'}, page_content='식당: 서울맛집\n리뷰: 가격대비 맛있게 푸짐하게\n먹을수있는것 같아요.'),
 Documen

In [ ]:
# 프롬프트
template = """
너는 {store} 식당의 리뷰들을 요약해주는 ai야.
리뷰들 요약해서 맛, 양, 가격, 서비스 부분에서 어떤지 줄글로 설명해줘.
"""
custom_prompt = PromptTemplate.from_template(template=template)
custom_prompt

PromptTemplate(input_variables=['store'], template='\n너는 {store} 식당의 리뷰들을 요약해주는 ai야.\n리뷰들 요약해서 맛, 양, 가격, 서비스 부분에서 어떤지 줄글로 설명해줘.\n')

### Ollama 사용

In [ ]:
llama = Ollama(
    model="llama3.1",
    temperature = 0.3
)

rag_chain = (
    {"context": openai_retriever, "store": RunnablePassthrough()}
    | custom_prompt
    | llama # 모델에 넣어서 답변 생성
)
query = "서울뚝배기"
response = rag_chain.invoke(query)

for chunk in response:
    print(chunk, end="", flush=True)

서울뚝배기는 한식 전문 레스토랑으로 다양한 한식料理를 제공합니다. 

**맛:**
서울뚝배기에서는 다양한 한식요리들이 제공되며, 고기가 풍부하고 맛이 좋습니다. 특히, 전복죽과 삼겹살은 인기 메뉴 중 하나입니다.

**양:**
음식 양은 충분하며, 가격대에 맞는 양을 제공합니다. 

**가격:**
서울뚝배기의 가격대는 일반적인 한식 레스토랑보다 조금 높습니다. 하지만, 고급한 한식 요리들을 제공하기 때문에 이해가 갑니다.

**서비스:**
서비스 또한 매우 좋으며, 친절하고 빠른 서비스를 제공합니다.

### gpt 사용

In [ ]:
gpt = OpenAI(
    model_name="gpt-4o-mini",
    temperature = 0.3
)

rag_chain = (
    {"context": openai_retriever, "store": RunnablePassthrough()}
    | custom_prompt
    | gpt # 모델에 넣어서 답변 생성
)
query = "서울뚝배기"
response = rag_chain.invoke(query)

for chunk in response:
    print(chunk, end="", flush=True)

C:\NLP\nlp\Lib\site-packages\langchain_community\llms\openai.py:253: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
C:\NLP\nlp\Lib\site-packages\langchain_community\llms\openai.py:1076: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


서울뚝배기 식당에 대한 리뷰를 종합해보면, 전반적으로 맛, 양, 가격, 서비스 측면에서 긍정적인 평가가 많습니다.

**맛**: 서울뚝배기의 음식은 깊고 진한 국물 맛이 특징이며, 재료의 신선함이 돋보입니다. 특히, 뚝배기 요리는 풍부한 풍미와 함께 따뜻한 온기를 제공하여 많은 손님들이 만족감을 표현하고 있습니다.

**양**: 제공되는 양은 대체로 푸짐하다는 평가가 많습니다. 특히, 뚝배기 메뉴는 한 끼 식사로 충분할 정도로 넉넉하게 나와서, 배고픈 손님들에게 좋은 선택이 됩니다.

**가격**: 가격대는 합리적이라는 의견이 많습니다. 맛과 양을 고려했을 때, 가성비가 뛰어난 편이라는 평이 주를 이루고 있습니다. 많은 손님들이 가격 대비 만족스러운 경험을 했다고 언급하고 있습니다.

**서비스**: 서비스 측면에서도 대체로 긍정적인 반응이 많습니다. 직원들은 친절하고, 신속하게 음식을 제공하며, 손님들의 요청에 잘 응대하는 편입니다. 이러한 서비스는 식사 경험을 더욱 즐겁게 만들어 주는 요소로 작용하고 있습니다.

종합적으로 볼 때, 서울뚝배기 식당은 맛, 양, 가격, 서비스 모두에서 좋은 평가를 받고 있으며, 많은 손님들이 재방문 의사를 밝히고 있습니다.

# 리뷰들을 프롬프트에 넣어주기

In [ ]:
# 50개의 리뷰를 넣는 프롬프트템플릿 만들기
template = """
너는 {store} 식당의 리뷰들을 요약해주는 ai야.
리뷰들 요약해서 맛, 양, 가격, 서비스 부분에서 어떤지 줄글로 설명해줘.
리뷰는 {text}야.
"""

query = '서울뚝배기'
template = PromptTemplate.from_template(template=template)
custom_prompt = template.format(store=query, text=[review for review in df.loc[df['식당']==query, '리뷰'].sample(50)])
custom_prompt

"\n너는 서울뚝배기 식당의 리뷰들을 요약해주는 ai야.\n리뷰들 요약해서 맛, 양, 가격, 서비스 부분에서 어떤지 줄글로 설명해줘.\n리뷰는 \n['해물 된장찌개 맛있음', '가성비 최고내요', '더웠음... 차라리 밖이 바람불어서 더 시원했음\\n전체적으로 음식이 자극적이지않았음', '점심시간에  자주감니다', '점심 메뉴는 심플하게\\n된장은 밥을 비벼먹을 수 있게 준비해 주셔서 좋슴니다~~', '벼르고 벼르다 드디어 다녀왔습니다. 역시 오래된 곳은 다 이유가 있네요. 반찬도 맛깔스럽고 제육도 자극적이지 않으면서 간이 알맞게 좋아요. 밥도둑처럼 순식간에 해치우고 나왔어요. 한번씩 들러서 모든 메뉴를 뽀개보려고 합니다. 너무너무 소중한 밥집이에요!', '혼밥하기좋은곳. 순두부 맛있어요', '회식장소 24시간', '맛있구요. 깨끗해요.', '직원분도 친절하시고 음식이 진짜 맛있어요!\\n도토리묵무침이랑 미역국 맛집입니다!', '점심은 5 종류의 메뉴에서 선택하는 것 같습니다.', '해장도 할수 있고, 점심 저녁 식사도 무난합니다.\\n휴일에 다른곳 다 문닫아도 열어서 감사하게 이용하고 있습니다 ㅎㅎ', '저렴하게 아침식사가 가능함. 2023년 2월 현재 식사 메뉴 9000원.', '밑반찬 다 맛있고 딴건 다 좋은데 파전은 너무 부실함.딱 그 가격에 그 퀄이라서 좀 아쉬움.', '싱싱한 야채로만 해물뚝배기 !  북어국! 미역국 !\\n반찬도 싱싱한 오이무침 !열무김치! 채장아치 !\\n완전집밥 같아요 ^~^ 강추!  강치 !', '맛도 좋고 가성비도 👍 👍 👍…', '점심메뉴에 한해서 육류를 좋아하는 분들은 별로 일수도 있습니다. 미역국 북어국 해물된장찌개 중에 하나밖에 주문이 안되기 때문이고 반찬도 거의 김치종류가 다라고 보시면 됩니다. 된장찌개 에는 비빔밥 재료가 나와서 비벼 드실 수 있고 찌개도 괜찮은 편입니다. 미역국 좋아하는 분들은 미역국이 맛있다고 하더라구요. 북어국도 괜찮긴 한데 매우 맛있는 건 아니고요. 인근 식당에서 가격대비 괜찮다 정도 입니다

### Ollama 이용

In [ ]:
llm = Ollama(
    model='llama3.1',
    temperature=0.3
)

response = llm.invoke(custom_prompt)
print(response)

for chunk in response:
    print(chunk, end="", flush=True)

**맛:**
해물된장 뚝배기, 해물된장찌개, 낙지볶음, 미역국, 해물아귀찜 등 다양한 메뉴의 맛이 좋다는 리뷰가 많습니다. 해물된장 뚝배기는 특히 강추되는 메뉴입니다.

**양:**
반찬 양이 적다는 리뷰도 있지만, 해물된장찌개와 같은 메뉴는 양이 충분하다고 리뷰하는 경우도 있습니다.

**가격:**
평균 가격은 7천원에서 8천원 사이이며, 점심 시간에는 인상되어 8000원에 판매되는 곳도 있습니다. 가격대가 합리적이라고 생각하는 리뷰도 있습니다.

**서비스:**
직원들의 친절함과 깔끔한 서비스는 리뷰의 일반적인 평점입니다.
**맛:**
해물된장 뚝배기, 해물된장찌개, 낙지볶음, 미역국, 해물아귀찜 등 다양한 메뉴의 맛이 좋다는 리뷰가 많습니다. 해물된장 뚝배기는 특히 강추되는 메뉴입니다.

**양:**
반찬 양이 적다는 리뷰도 있지만, 해물된장찌개와 같은 메뉴는 양이 충분하다고 리뷰하는 경우도 있습니다.

**가격:**
평균 가격은 7천원에서 8천원 사이이며, 점심 시간에는 인상되어 8000원에 판매되는 곳도 있습니다. 가격대가 합리적이라고 생각하는 리뷰도 있습니다.

**서비스:**
직원들의 친절함과 깔끔한 서비스는 리뷰의 일반적인 평점입니다.

### gpt 사용

In [ ]:
gpt = OpenAI(
    model_name="gpt-4o-mini",
    temperature = 0.3
    )


response = gpt.invoke(custom_prompt)

for chunk in response:
    print(chunk, end="", flush=True)

C:\NLP\nlp\Lib\site-packages\langchain_community\llms\openai.py:253: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
C:\NLP\nlp\Lib\site-packages\langchain_community\llms\openai.py:1076: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


서울뚝배기 식당에 대한 리뷰를 종합해보면, 전반적으로 맛, 양, 가격, 서비스 모두 긍정적인 평가를 받고 있습니다.

**맛**: 해물된장찌개와 아귀찜, 제육볶음 등 다양한 메뉴가 맛있다는 의견이 많습니다. 특히 해물된장찌개는 신선한 해산물과 진한 된장 맛이 잘 어우러져 있다는 평이 많고, 미역국과 밑반찬도 정갈하고 맛있다는 언급이 있습니다. 반면, 김치전은 별로라는 의견도 있어 메뉴에 따라 차이가 있을 수 있습니다.

**양**: 양에 대한 평가는 대체로 긍정적입니다. 밥이 푸짐하게 제공되며, 찌개와 반찬도 적당히 나와서 만족스럽다는 의견이 많습니다. 특히, 해물된장찌개와 함께 나오는 밥은 '밥도둑'이라는 표현으로 그 양과 맛을 강조하는 리뷰도 있었습니다.

**가격**: 가격은 대체로 저렴하다는 평가를 받고 있습니다. 해물된장찌개가 7천 원으로 가성비가 뛰어난 편이며, 점심 메뉴도 8천 원으로 합리적인 가격에 제공된다는 의견이 많습니다. 오랜 역사를 가진 대중음식점으로서 가격 대비 맛이 좋다는 점이 강조됩니다.

**서비스**: 서비스에 대한 평가는 대체로 긍정적입니다. 직원들이 친절하다는 의견이 많고, 음식이 빠르게 제공된다는 점도 언급됩니다. 그러나 일부 리뷰에서는 혼잡한 시간대에 서비스가 다소 미흡할 수 있다는 점도 지적되었습니다.

종합적으로 볼 때, 서울뚝배기 식당은 맛과 양, 가격, 서비스 모두에서 좋은 평가를 받고 있는 곳으로, 특히 해물된장찌개와 아귀찜이 인기 메뉴로 자리 잡고 있습니다.

# 최종

In [ ]:
# 50개의 리뷰를 넣는 프롬프트템플릿 만들기
template = """
너는 {store} 식당의 리뷰들을 요약해주는 ai야.
리뷰들 요약해서 맛, 양, 가격, 서비스 부분에서 어떤지 줄글로 설명해줘.
리뷰는 {text}야.
"""

query = '서울뚝배기'
template = PromptTemplate.from_template(template=template)
custom_prompt = template.format(store=query, text=[review for review in df.loc[df['식당']==query, '리뷰'][:100]])
custom_prompt

gpt = OpenAI(
    model_name="gpt-4o-mini",
    temperature = 0.3
    )


response = gpt.invoke(custom_prompt)

for chunk in response:
    print(chunk, end="", flush=True)

C:\NLP\nlp\Lib\site-packages\langchain_community\llms\openai.py:253: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
C:\NLP\nlp\Lib\site-packages\langchain_community\llms\openai.py:1076: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


서울뚝배기 식당에 대한 리뷰를 종합해보면, 전반적으로 맛, 양, 가격, 서비스 측면에서 긍정적인 평가가 많습니다.

**맛**: 식당에서 제공하는 주요 메뉴인 미역국, 된장찌개, 순두부찌개, 북어국은 대체로 맛있다는 평가를 받고 있습니다. 특히 미역국과 해물된장찌개는 깊은 맛을 자랑하며, 반찬들도 신선하고 맛있다는 의견이 많습니다. 다만, 일부 리뷰에서는 특정 메뉴가 평범하다는 언급도 있어, 개인의 취향에 따라 다소 차이가 있을 수 있습니다.

**양**: 제공되는 양은 적당하다는 평가가 많으며, 특히 비빔밥과 함께 나오는 찌개는 양이 충분하다는 의견이 있습니다. 반찬도 여러 종류가 제공되어 식사를 더욱 풍성하게 만들어줍니다.

**가격**: 가격은 대체로 합리적이라는 평가가 많습니다. 대부분의 리뷰에서 9,000원이라는 가격이 만족스럽다는 의견이 있으며, 가성비가 뛰어난 식당으로 언급되고 있습니다. 특히 아침식사 메뉴는 저렴한 가격에 제공되어 많은 손님들이 찾는 이유 중 하나입니다.

**서비스**: 서비스는 신속하고 친절하다는 평가가 많습니다. 직원들이 손님을 잘 응대하며, 음식이 빨리 나오는 점이 특히 긍정적으로 언급되고 있습니다. 다만, 일부 리뷰에서는 서비스의 친절함이 부족하다는 의견도 있어, 일관성이 다소 떨어질 수 있습니다.

종합적으로 서울뚝배기 식당은 맛과 가격, 서비스 측면에서 만족스러운 경험을 제공하는 곳으로, 특히 간단하고 건강한 한식을 원하시는 분들에게 추천할 만한 식당입니다.

In [ ]:
df.loc[df['식당']=='서울뚝배기', ['리뷰']][:100]

,리뷰
802,This shop only offers 4 options - seaweed soup...
803,벼르고 벼르다 드디어 다녀왔습니다. 역시 오래된 곳은 다 이유가 있네요. 반찬도 맛...
804,"여기 음식 다 진국(맛있음)\n\n밑반찬 맛있고 신선함.\n\n된장찌개, 미역국, ..."
805,점심에 인기있는 퇴계로 한식식당입니다.\n해물된장찌개 아주 좋아요. 북어국도 최고~...
806,소박한 해물된장뚝배기에 곁들여 나오는 야채 비빔밥이 맛도 좋고 건강에도 좋을 것 같...
...,...
897,집밥 같아요. 안질리는 맛입니다. 가세요.
898,반찬이 깔끔하고 빨리 나와요
899,건강한 맛!식당 이모님들이 착하세요!
900,점심시간에 자주감니다


In [ ]:
import gradio as gr